In [1]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
from flask import Flask, jsonify, request, url_for, redirect, render_template
import datetime as dt
import os
import csv 
import numpy as np
from flask_pymongo import PyMongo
import lda
import lda.datasets

In [11]:
data = pd.read_csv("2013-06 - Citi Bike trip data.csv", index_col=None)
data1 = pd.read_csv("2013-07 - Citi Bike trip data.csv", index_col=None)
data2 = pd.read_csv("2013-08 - Citi Bike trip data.csv", index_col=None)

In [4]:
data1 = pd.read_csv("2013-07 - Citi Bike trip data.csv", index_col=None)


,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,634,2013-07-01 00:00:00,2013-07-01 00:10:34,164,E 47 St & 2 Ave,40.753231,-73.970325,504,1 Ave & E 15 St,40.732219,-73.981656,16950,Customer,\N,0
1,1547,2013-07-01 00:00:02,2013-07-01 00:25:49,388,W 26 St & 10 Ave,40.749718,-74.002950,459,W 20 St & 11 Ave,40.746745,-74.007756,19816,Customer,\N,0
2,178,2013-07-01 00:01:04,2013-07-01 00:04:02,293,Lafayette St & E 8 St,40.730287,-73.990765,237,E 11 St & 2 Ave,40.730473,-73.986724,14548,Subscriber,1980,2
3,1580,2013-07-01 00:01:06,2013-07-01 00:27:26,531,Forsyth St & Broome St,40.718939,-73.992663,499,Broadway & W 60 St,40.769155,-73.981918,16063,Customer,\N,0
4,757,2013-07-01 00:01:10,2013-07-01 00:13:47,382,University Pl & E 14 St,40.734927,-73.992005,410,Suffolk St & Stanton St,40.720664,-73.985180,19213,Subscriber,1986,1
5,861,2013-07-01 00:01:23,2013-07-01 00:15:44,511,E 14 St & Avenue B,40.729387,-73.977724,454,E 51 St & 1 Ave,40.754557,-73.965930,16223,Subscriber,1988,1
6,550,2013-07-01 00:01:59,2013-07-01 00:11:09,293,Lafayette St & E 8 St,40.730287,-73.990765,394,E 9 St & Avenue C,40.725213,-73.977688,16746,Customer,\N,0
7,288,2013-07-01 00:02:16,2013-07-01 00:07:04,224,Spruce St & Nassau St,40.711464,-74.005524,376,John St & William St,40.708621,-74.007222,16062,Subscriber,1985,2
8,766,2013-07-01 00:02:16,2013-07-01 00:15:02,432,E 7 St & Avenue A,40.726218,-73.983799,336,Sullivan St & Washington Sq,40.730477,-73.999061,17963,Subscriber,1980,2
9,773,2013-07-01 00:02:23,2013-07-01 00:15:16,173,Broadway & W 49 St,40.760647,-73.984427,479,9 Ave & W 45 St,40.760193,-73.991255,19365,Subscriber,1989,1


In [6]:
data.describe()

,tripduration,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,bikeid,birth year,gender
count,5.777030e+05,577703.000000,577703.000000,577703.000000,559644.000000,559644.000000,559644.000000,577703.000000,337382.000000,577703.000000
mean,1.372570e+03,439.228836,40.732436,-73.991251,441.910624,40.732146,-73.991244,17617.434516,1974.644945,0.710431
std,9.028063e+03,337.240318,0.021423,0.013002,345.448311,0.021507,0.013105,1677.518719,10.780255,0.678268
min,6.100000e+01,72.000000,40.646607,-74.017134,72.000000,40.646607,-74.017134,14529.000000,1899.000000,0.000000
25%,5.140000e+02,309.000000,40.717400,-74.001768,309.000000,40.716887,-74.001971,16161.000000,1968.000000,0.000000
50%,8.740000e+02,404.000000,40.734546,-73.990741,403.000000,40.734232,-73.990741,17638.000000,1977.000000,1.000000
75%,1.398000e+03,482.000000,40.749013,-73.981948,482.000000,40.748549,-73.981923,19066.000000,1983.000000,1.000000
max,3.876479e+06,3020.000000,40.771522,-73.950048,3020.000000,40.771522,-73.950048,20625.000000,1997.000000,2.000000


In [7]:
data1.describe()

,tripduration,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,bikeid,gender
count,8.434160e+05,843416.000000,843416.000000,843416.000000,843416.000000,843416.000000,843416.000000,843416.000000,843416.000000
mean,9.981007e+02,428.445424,40.733611,-73.991241,429.437272,40.733218,-73.991257,17615.627889,0.978747
std,7.146790e+03,313.478523,0.020584,0.012760,320.087328,0.020652,0.012888,1703.568990,0.626986
min,6.000000e+01,72.000000,40.680342,-74.017134,72.000000,40.680342,-74.017134,14529.000000,0.000000
25%,4.440000e+02,305.000000,40.718939,-74.001547,304.000000,40.717821,-74.001768,16136.000000,1.000000
50%,7.290000e+02,401.000000,40.735324,-73.990931,399.000000,40.734927,-73.990931,17627.000000,1.000000
75%,1.202000e+03,482.000000,40.750200,-73.981923,480.000000,40.749718,-73.981923,19088.000000,1.000000
max,6.250750e+06,3002.000000,40.770513,-73.950048,3002.000000,40.770513,-73.950048,20626.000000,2.000000


In [12]:
data2.describe()

,tripduration,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,bikeid,gender
count,1.001958e+06,1.001958e+06,1.001958e+06,1.001958e+06,1.001958e+06,1.001958e+06,1.001958e+06,1.001958e+06,1.001958e+06
mean,9.795802e+02,4.402721e+02,4.073396e+01,-7.399122e+01,4.406920e+02,4.073360e+01,-7.399126e+01,1.759646e+04,1.012827e+00
std,2.258803e+03,3.486319e+02,2.031681e-02,1.269275e-02,3.540579e+02,2.040072e-02,1.280682e-02,1.699679e+03,6.197801e-01
min,6.000000e+01,7.200000e+01,4.068034e+01,-7.401713e+01,7.200000e+01,4.068034e+01,-7.401713e+01,1.452900e+04,0.000000e+00
25%,4.400000e+02,3.040000e+02,4.071911e+01,-7.400155e+01,3.040000e+02,4.071894e+01,-7.400177e+01,1.612300e+04,1.000000e+00
50%,7.240000e+02,4.020000e+02,4.073588e+01,-7.399093e+01,4.010000e+02,4.073532e+01,-7.399093e+01,1.759100e+04,1.000000e+00
75%,1.199000e+03,4.840000e+02,4.075020e+01,-7.398195e+01,4.820000e+02,4.074972e+01,-7.398195e+01,1.905500e+04,1.000000e+00
max,7.617930e+05,3.002000e+03,4.077051e+01,-7.395005e+01,3.002000e+03,4.077051e+01,-7.395005e+01,2.066400e+04,2.000000e+00
